# Dashboard

In [1]:
#pip install jupyter-dash

In [2]:
import pandas as pd
from dash import Dash, dcc, html

In [3]:
app = Dash(__name__)

app.layout = html.Div([
    html.Div(children='Hello World')
])

In [4]:
app.run_server(host='0.0.0.0', port=8050, debug=True)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/debugger.py", line 6, in <module>
    from IPython.core.completer import IPCompleter
  File "/home/ciara/anaconda3/

SystemExit: 1

/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/terminal/debugger.py", line 6, in <module>
    from IPython.core.completer import IPCompleter
  File "/home/ciara/anaconda3/

SystemExit: 1

/home/ciara/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
#Read in the ccombined data
finnish = pd.read_csv("finnish_data_combined.csv")
irish = pd.read_csv("irish_data_combined.csv")
irish.pop("Unnamed: 0")
finnish.pop("Unnamed: 0")

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
Name: Unnamed: 0, dtype: int64

In [7]:
finnish.head()

,Year,Total,Labour,Materials,Services,Residential buildings completion (No.)
0,1995,103.5,103.0,105.3,99.1,24526
1,1996,102.7,103.5,104.3,96.3,21372
2,1997,105.2,106.9,107.2,96.8,26955
3,1998,107.6,109.4,109.8,98.5,30529
4,1999,109.1,113.5,110.2,99.0,30537


In [8]:
irish.head()

,Quarter,House Construction Cost Index,No. of Units
0,q1_2018,212.513233,14816.0
1,q1_2019,214.525667,20324.0
2,q1_2021,218.490518,34511.0
3,q2_2018,213.003957,16351.0
4,q2_2019,215.117876,22139.0


In [12]:
app = Dash(__name__)

In [15]:
# app.py

# ...

app.layout = html.Div(

    children=[

        html.H1(children="Building construction in Ireland and Finland"),

        html.P(

            children=(

                "Analyze the behavior of avocado prices and the number"

                " of avocados sold in the US between 2015 and 2018"

            ),

        ),

        dcc.Graph(

            figure={

                "finnish": [

                    {

                        "x": finnish["Total"],

                        "y": finnish["Residential buildings completion (No.)"],

                        "type": "lines",

                    },

                ],

                "layout": {"title": "Finnish total buildings completed vs cost index of construction"},

            },

        ),

        dcc.Graph(

            figure={

                "irish": [

                    {

                        "x": irish["House Construction Cost Index"],

                        "y": irish["No. of Units"],

                        "type": "lines",

                    },

                ],

                "layout": {"title": "Irish total buildings completed vs cost index of construction"},

            },

        ),

    ]

)

In [16]:
# app.py

# ...

if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 98] Address already in use

## References

https://realpython.com/python-dash/